<a href="https://colab.research.google.com/github/The-Sch-O/EI-WebSearch/blob/main/Jules_day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Information Retrieval
## Introduction to search engines

### DAY 2: Teacher version
### Implementing a search engine

The goal of this second session is to implement a first architecture of a search engine on the previously introduced dataset (stackexchange-datascience). If you missed the first session or if you did not saved the dataset, please reload the first session's notebook to download it. 

If you need some ifnormation about the dataset, it should be available here : https://archive.org/details/stackexchange

The notebook is divided into several steps:
-	Implement the indexation
-	Implement the search method
-	Define a ranking strategy and implement it
-	Suggest some improvements of the search engine



## Initialisation

In [ ]:
!pip install ttable


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.3/122.3 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ttable: filename=ttable-0.6.4-cp310-cp310-linux_x86_64.whl size=212625 sha256=b6297a4d5f537f4debb1a98a4a5838711bf0a0bd6482696a63bf774b1ddb4f47
  Stored in directory: /root/.cache/pip/wheels/0d/8d/56/f2572fdbf1ef1f8a947d7ff25ce18d9373d8e02a68f9ac8de6
Successfully built ttable


In [2]:
import pandas as pd
import re
import os
import math
import numpy as np
from sklearn.metrics import mean_squared_error
from itertools import product
import numpy as np

In [ ]:
!python --version

Python 3.10.11


In [4]:
# Only if you use Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# TODO:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = '/content/drive/MyDrive/EI_ST4/data' 
print(DATA_PATH)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/EI_ST4/data


**Important :**

An Excel file for testing the evaluation part is available in the gitlab repo : evaluation_search_engine_post_queries_ranking_EI_CS.xlsx

If you work on Colab, we advice you to push it directly on your Google Drive directory.

# Implement the indexation
As you might already know, for a search engine to work properly an index of the documents must be created. Here we will keep it in python, and try to use only common libraries to keep it simple.

Once created, the index will be used to match the query with the documents. As a result, there are several ways to build an index, using statistical, boolean, semantic indexation...

First of, let's make a naive one that will consist in breaking down each document into a set of the words it contains.

In [35]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.4 MB/s eta 0:00:00


In [40]:
import nltk
nltk.download('wordnet')
import transformers

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [36]:
"""from nltk.tokenize import RegexpTokenizer

def extract_words(text:str)->list:
  # 2. On extrait les mots et les nombres
  tokenizer = RegexpTokenizer('[a-zA-Z]{2,}|\d+\S?\d*')
  wordtokens = tokenizer.tokenize(text)

  return text
"""
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer
def extract_words(text):
  RES=[] #liste des termes qui seront ensuite lemmatisés

  if len(text)==0:
    pass
  else:
    lemmatizer = WordNetLemmatizer()
    tokenizer=AutoTokenizer.from_pretrained('bert-base-cased') 
    res=tokenizer.tokenize(text) #liste de mots tokenizé
    for x in res:
      y=lemmatizer.lemmatize(x)
      RES.append(y)
    
  
  return RES


In [ ]:
extract_words("I would like to have a swimming pool")


['I', 'would', 'like', 'to', 'have', 'a', 'swimming', 'pool']

In [ ]:

# test
s = "The cat is sat on the mat. The dog is laid on the mat."
assert extract_words(s).sort()==["The","cat","is","sat","on","the","mat.","dog","laid"].sort()

False

As you may notice, there are several problems with the previous implementation. First, "The" and "the" aren't considered the same, the "." is kept at the the end of "mat." as any other punctuation character... 

Re-implement this function with some basic preprocessing to avoid these issues.

In [ ]:
# test
assert extract_words(s).sort()==["the","cat","is","sat","on","mat","dog","laid"].sort()

Now you sould be able to create your index table. For now we will just make a dataframe with two columns: [raw_text, words].

In [ ]:
# test

L = [s, "Hello World!", "Goodbye", "How are you?"]

index_docs(L)

on va extraire
The cat is sat on the mat. The dog is laid on the mat.
------
on va extraire
Hello World!
------
on va extraire
Goodbye
------
on va extraire
How are you?
------


,raw_text,words
0,The cat is sat on the mat. The dog is laid on ...,"[The, cat, is, sat, on, the, mat, ., The, dog,..."
1,Hello World!,"[Hello, World, !]"
2,Goodbye,[Goodbye]
3,How are you?,"[How, are, you, ?]"


In [ ]:
print(os.path.join(DATA_PATH, 'Posts.xml'))

/content/drive/MyDrive/EI_ST4/data/Posts.xml


Now, let's try it on the dataset:

In [ ]:
posts = pd.read_xml(os.path.join(DATA_PATH, 'Posts.xml'), parser="etree", encoding="utf8")


For our first version of the indexation mechanism, we will simply use the "body" of the posts. To have a better search engine, the title and other metadata aswell could be used aswell. Finally, not all the XML files have a "body" feature, so for the search engine to retrieve information from any of the files you will need to implement another way to index.

But first, let's start with "body". There is more to preprocess than before, indeed, there are html tags such as "<p>" for instance. They are not useful for us, because users won't use them in their queries. So we first need to remove them.

In [ ]:

import re
def remove_tags(text:str)->str:
  # TODO
  clean_text = text.replace('\n', ' ')
  #clean_text = ' '.join(c_text.split())


  pattern = re.compile('<.*?>')


  result = re.sub(pattern, '', clean_text)

  return result

In [ ]:
# test
remove_tags('<p>Hello World!\nI am making a search engine.<p>')

'Hello World! I am making a search engine.'

In [ ]:
clean_posts = posts[['Id','Body']]
clean_posts['Clean Body'] = clean_posts['Body'].fillna('').apply(remove_tags)
clean_posts

In [ ]:
clean_posts['words'] = clean_posts['Clean Body'].apply(extract_words)
clean_posts

## Zipf Law

A way of analyzing a corpus is to draw the zipf law

In [ ]:
# TODO : Draw Zipf Law on the Posts Corpus

## Inverted Index

Now, we want to go further on the indexing and build an inverted index. Inverted index is a dictionary where the keys are the words of the vocabulary and the values are the documents containing these words. Reducing the size of the vocabulary is a relevant first step when building an inverted index. Here, we will focus on the creation of the index, we leave you the optimisation steps :)

In [ ]:
import pandas as pd

def index_docs(docs:list[str])->pd.DataFrame:
  df=pd.DataFrame()
  phrases=[]
  Mots=[]

  for p in L:
    print('on va extraire')
    mots=extract_words(p)
    print(p)
    print('------')
    phrases.append(p)
    Mots.append(mots)
  df['raw_text']=phrases
  df['words']=Mots
  print(Mots)
  return df

In [ ]:
import pandas as pd
def create_index(posts:pd.DataFrame)-> set: #on suppose posts déjà traité
  inv_ind = {}
  for i in range(len(posts['Id'])):
    words = posts['words'][i]; id = posts['Id'][i]#liste des mots du doc i ; 
    for word in words:
      if word in inv_ind:
        inv_ind[word] += [id]
      else:
        inv_ind[word] = [id]
  return inv_ind

In [ ]:
inverted_index = create_index(clean_posts.iloc[0:5000])

NameError: ignored

#### Well Done, you've indexed the dataset! 
Don't hesitate to save your indexes in txt or pickle file

---
# Implement the search method

A naive method would be to count the number of words in common between the query and each posts. Then to rank the posts you could directly select the post who maximize the number of common words. Let's implement this approach :

In [ ]:
# Implement the word_in_index function 
# Inputs : a word (str) & a list of words
# Output : pandas series of 1 if the word is in the list, else 0

def word_in_index(word, word_list_index):
  # TODO

  return

In [ ]:
# Implement the function which run through a pandas series and count the number of word in common
# Use extract_words method, apply method with word_in_index function
# Inputs : the query (str) & pandas series of strings
# Output : Pandas series counting the number of common words between the query and each string in word_serie

def count_common_words(query, word_serie):
  # TODO
  
  return


In [ ]:

def rank_top_query(query, df, top=5):
  # TODO


  return 

In [ ]:
rank_top_query(query="testing the query in python", df=clean_posts, top=5)

Testez plusieurs requêtes et critiquez les résultats obtenus.

Quels sont les pros and cons de cette méthodes. Vous l'indiquerez sur le rapport avec vos réflexions pour l'améliorer.

Next, you have to implement the first improvements you find in the search method to get most relevant results 

## Boolean Search

Thanks to the ttable library, implement a boolean search method

In [ ]:
!python --version


Python 3.10.11


In [12]:
!pip install ttable


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.3/122.3 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ttable: filename=ttable-0.6.4-cp310-cp310-linux_x86_64.whl size=212610 sha256=37a0115d0c0bb33ea36bf189478611c090d7774418497d119d5eb97fe25347ca
  Stored in directory: /root/.cache/pip/wheels/0d/8d/56/f2572fdbf1ef1f8a947d7ff25ce18d9373d8e02a68f9ac8de6
Successfully built ttable


In [14]:
from tt import BooleanExpression

In [ ]:
!apt install ttable


In [7]:
import pickle

# Load the pickled file
with open(os.path.join(DATA_PATH,"inverted_index.pickle"), "rb") as file:
    loaded_dict = pickle.load(file)

# Access the loaded dictionary



In [ ]:
!pip install Tokeni

In [ ]:
b

In [ ]:
"""inv_index_simple = {}
for word in loaded_dict:
  l=[]
  tuple_list = loaded_dict[word]['inv_ind']
  for elt in tuple_list:
    (doc_id,_)=elt #elt = (a,b)
    l.append(doc_id)
  inv_index_simple[word]=l"""
inv_index_simple['learning']



In [42]:
def boolean_search(query):#the query est sous la formenormale conjonctive A1 OR A2 OR A3 OR A4...
  query_extracted = extract_words(query)
#transforme la requête en booléen
def transformation_query_to_boolean(query):
    boolean_query=[]
    for token in query.split():
        boolean_query.append(token)
        boolean_query.append('AND')
    boolean_query.pop()
    return boolean_query
from tt import BooleanExpression
BooleanOperator = {"AND", "OR", "NOT"}
def transformation_query_to_postfixe(query):
    b = BooleanExpression(query)
    return b.postfix_tokens
#merge deux posting lists selon l'opérateur
def merge_and_postings_list(posting_term1,posting_term2):
    result=[]
    n = len(posting_term1)
    m = len(posting_term2)
    i = 0
    j = 0
    while i < n and j <m:
        if posting_term1[i] == posting_term2[j]:
            result.append(posting_term1[i])
            i = i+1
            j = j+1
        else:
            if posting_term1[i] < posting_term2[j]:
                i = i+1
            else:
                j=j+1
    return result
def merge_or_postings_list(posting_term1,posting_term2):
    result=[]
    n = len(posting_term1)
    m = len(posting_term2)
    i = 0
    j = 0
    while i < n and j <m:
        if posting_term1[i] == posting_term2[j]:
            result.append(posting_term1[i])
            i = i+1
            j = j+1
        else:
            if posting_term1[i] < posting_term2[j]:
                result.append(posting_term1[i])
                i = i+1
            else:
                result.append(posting_term2[j])
                j=j+1
    if i <n:
        result = result + posting_term1[i:]
    if j <m:
        result = result + posting_term2[j:]
    return result


def merge_and_not_postings_list(posting_term1,posting_term2):
    result=[]
    n = len(posting_term1)
    m = len(posting_term2)
    i = 0
    j = 0
    while i < n and j <m:
        if posting_term1[i] == posting_term2[j]:
            i = i+1
            j = j+1
        else:
            if posting_term1[i] < posting_term2[j]:
                result.append(posting_term1[i])
                i = i+1
            else:
                j=j+1
    return result
#généralise le merge selon l'opérateur
def boolean_operator_processing_with_inverted_index(BoolOperator,posting_term1,posting_term2):
    result=[]
    if BoolOperator == "AND":
        result.append(merge_and_postings_list(posting_term1,posting_term2))
    elif BoolOperator=="OR" :
        result.append(merge_or_postings_list(posting_term1,posting_term2))
    elif BoolOperator == "NOT":
        result.append(merge_and_not_postings_list(posting_term1,posting_term2))
    return result
def processing_boolean_query_with_inverted_index(booleanOperator,query, inverted_index):
    evaluation_stack = []
    #transformer query en liste de mots
    """res = word_tokenize(query)
    Docs_id = []
    for i in range(len(res)):
      res[i] = lemmatizer.lemmatize(res[i])"""
    query=extract_words(query)

    for term in query:
        if term.upper() not in booleanOperator:
          """print(term.upper())
          print(inverted_index[term.upper()])"""
          evaluation_stack.append(inverted_index[term.upper()])#on rajoute la posting list du dernier terme
        else:
            if term.upper() == "NOT":
              operande= evaluation_stack.pop()
              eval_prop = boolean_operator_processing_with_inverted_index(term.upper(), evaluation_stack.pop(),operande)
              evaluation_stack.append(eval_prop[0])
                #evaluation_stack.append(eval_prop[0])#pq 2 fois?
            else:
              operator = term.upper()
              eval_prop =  boolean_operator_processing_with_inverted_index(operator, evaluation_stack.pop(),evaluation_stack.pop())
              evaluation_stack.append(eval_prop[0])
    return  evaluation_stack.pop()
def compare(inverted_index,boolean_query_processed):
  pass



In [43]:
processing_boolean_query_with_inverted_index(BooleanOperator,'machine learning',inv_index_simple)

[8984,
 9221,
 17745,
 23857,
 24579,
 24875,
 37803,
 40086,
 42170,
 43241,
 54674,
 72523,
 81150,
 93013,
 104167,
 104329,
 104489,
 107475,
 113423,
 118409,
 118665]

## Probabilistic search

Implement the MIB or BM25 method of searching

In [ ]:
def probabilistic_search(query):
  # TODO

  return

Compare the naive method with your improvements and the boolean and probabilistic search. (report)



---



---




# Evaluate the Search

Now you implement multiple search methods and you're able to improve it. You have to define metric to compare it objectively.



We ask you to implement NDCG (Normalized Discounted Cumulative Gain) from few queries we implement on a dozen of post. We already defined the values of relevance judgement in the xlsx file : . The final score will be the mean quadratic error of the queries.


Explication for the xlsx file :

We propose you a Excel file with some posts and a mesure of relevancy for the queries

- First column is the post Id,
- Columns starting by query are the queries you have to test.
- The values in this columns are the rank of relevancy of the post in regard with the query.
- The missing values indicates you should not take into account the post


You will have to criticize this metric and your result in the report. Then you will have to propose some improvements. 

Thereafter in this week, you will have to compare your different search engines.

In [ ]:
# Read Relevancy CSV
df_relevancy = pd.read_excel("/content/drive/MyDrive/TP Centrale/evaluation_search_engine_post_queries_ranking_EI_CS.xlsx")

In [ ]:
def calculate_ndgc(query_col="query", output_col="query_output"):
  # TODO

  return



In [ ]:
#Pistes d'amélioration
import pandas as pd
import os
from google.colab import drive

drive.mount('/content/drive')
DATA_PATH = '/content/drive/MyDrive/EI_ST4/data' 
posts = pd.read_xml(os.path.join(DATA_PATH, 'Posts.xml'), parser="etree", encoding="utf8")
from google.colab import drive



Mounted at /content/drive


In [ ]:
posts.head()
df=posts[['Id','ViewCount','AnswerCount','CommentCount','FavoriteCount','ClosedDate']]
df.head()

,Id,ViewCount,AnswerCount,CommentCount,FavoriteCount,ClosedDate
0,5,898.0,1.0,1,NaN,2014-05-14T14:40:25.950
1,7,478.0,3.0,4,NaN,2014-05-14T08:40:54.950
2,9,NaN,NaN,0,NaN,None
3,10,NaN,NaN,1,NaN,None
4,14,1901.0,4.0,1,NaN,None


In [ ]:
c1 = 1
c2 = 1
df=posts[['Id','ViewCount','AnswerCount','CommentCount','FavoriteCount','ClosedDate']]
scores=[[] for k in range (len(df))]
for index, row in df.iterrows():

  scores[index] = [row['ViewCount']]
  scores[index].append(  row['AnswerCount']) #mais on veut normaliser tout ça
#on normalise les scores:
import numpy as np
scores=np.array(scores)
for s in scores:
  s[0]=s[0]/np.amax(scores[:,0])*c1
  s[1]=s[1]/np.amax(scores[:,1])*c2


In [ ]:
listes=[[1,2],[2,4],[3,5]]
import numpy as np
listes=np.array(listes)
print(listes[:,1])
print(np.amax(listes[:,1]))

In [ ]:
import pandas as pd
df=pd.DataFrame()

df['c1']=['Jules','Zenta','Joshua']
df['id']=['11','12',13]

In [ ]:
def classement(list_relevant_docs,dico):
  pass